# Workflow

## 1.1 - Import Packages

In [ ]:
import importlib.metadata

packages = [
    "fastapi",
    "requests",
    "pywav",
    "redis",
    "uuid",
]

for package_name in packages:
    try:
        version = importlib.metadata.version(package_name)
        print(f"'{package_name}' is already installed (version {version}).")
    except importlib.metadata.PackageNotFoundError:
        print(f"'{package_name}' is not installed. Installing now...")
        # Attempt to install the package
        ! pip install "{package_name}"

## 1.2 - Redis Ping

In [ ]:
import redis

redis_client = redis.Redis(host="localhost", port=6379, decode_responses=True)
redis_client.ping()

## 1.3 - Generate random UUID

In [ ]:
from uuid import uuid4

redis_id = uuid4()
print(redis_id)

# 2.0 - Redis Channels

### 2.1 - Create a Redis Channel

In [ ]:
from redis_pool import RedisPoolProvider, publish
from fastapi import Depends

redis_client = RedisPoolProvider()


async def create_channel(redis_client: RedisPoolProvider = Depends(RedisPoolProvider)):
    channel = f"{redis_id}:jupyter"
    await publish(redis_client, channel, "establish channel")


await create_channel(redis_client)

### 2.2 - Publish Messages to Redis Channel

In [ ]:
async def publish_message(redis_client: RedisPoolProvider = Depends(RedisPoolProvider)):
    channel = f"{redis_id}:jupyter"
    await publish(redis_client, channel, "test message")


await publish_message(redis_client)

### 2.3 - Subscribe to Redis Channel

In [ ]:
from redis_pool import RedisPoolProvider, subscribe
from fastapi import Depends


async def create_subscriber(
    redis_client: RedisPoolProvider = Depends(RedisPoolProvider),
):
    channel = f"{redis_id}:jupyter"
    msg_queue = []
    async for msg in subscribe(channel, redis_client):
        msg_queue.append(msg)

    return msg_queue


await create_subscriber(redis_client)